In [27]:
import sys
sys.path.insert(0, 'ladybug')
sys.path.insert(0, 'honeybee')

import json
import numpy as np
import json

from honeybee.energyplus.filemanager import get_energy_plus_objects_from_file
from honeybee._hbanalysissurface import HBAnalysisSurface

from honeybee.radiance.material.glass import Glass
from honeybee.radiance.material.plastic import Plastic

In [16]:
def load_json(path):
    """
    Description:
        Load a JSON file into a dictionary object
    Arguments:
        path [string]: The location of the JSON file being loaded
    Returns:
        dictionary [dict]: Dictionary containing contents of loaded JSON file
    """
    with open(path) as data_file:
        return json.load(data_file)


def unit_vector(vector):
    """
    Description:
        Returns the unit vector of a vector
    Arguments:
        vector [1D-array]: Vector defined as [n.n, n.n, n.n]
    Returns:
        vector [1D-array]: Dictionary containing contents of loaded JSON file
    """
    return vector / np.linalg.norm(vector)


def angle_between(vector_1, vector_2):
    """ Returns the angle in radians between vectors 'vector_1' and 'vector_2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    vector_1_u = unit_vector(vector_1)
    vector_2_u = unit_vector(vector_2)
    return np.arccos(np.clip(np.dot(vector_1_u, vector_2_u), -1.0, 1.0))


def rotate(origin, point, angle):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in radians.
    """
    o_x, o_y = origin
    p_x, p_y = point

    q_x = o_x + np.cos(angle) * (p_x - o_x) - np.sin(angle) * (p_y - o_y)
    q_y = o_y + np.sin(angle) * (p_x - o_x) + np.cos(angle) * (p_y - o_y)
    return q_x, q_y

In [17]:
# Specify the config file to be referenced
CONFIG = load_json("idf_config.json")

# Load the EnergyPlus file objects
EP_FILE_PATH = CONFIG["target_idf"]
IDF = get_energy_plus_objects_from_file(EP_FILE_PATH)

EPW_FILE = CONFIG["weather_file"]

In [18]:
# Set the "vector to north", so that wall orientation can be obtained
IDF_ANGLE_TO_NORTH = np.radians(float(IDF["building"]["IDFName"][2]))
IDF_VECTOR_TO_NORTH = (np.sin(IDF_ANGLE_TO_NORTH), np.cos(IDF_ANGLE_TO_NORTH), 0)

In [22]:
# Define materials to be applied to surfaces
glass_material_N = Glass(
    "GlassMaterialN",
    r_transmittance=CONFIG["glass_visible_transmittance_N"],
    g_transmittance=CONFIG["glass_visible_transmittance_N"],
    b_transmittance=CONFIG["glass_visible_transmittance_N"],
    refraction_index=1.52
    )

glass_material_S = Glass(
    "GlassMaterialS",
    r_transmittance=CONFIG["glass_visible_transmittance_S"],
    g_transmittance=CONFIG["glass_visible_transmittance_S"],
    b_transmittance=CONFIG["glass_visible_transmittance_S"],
    refraction_index=1.52
)

glass_material_E = Glass(
    "GlassMaterialE",
    r_transmittance=CONFIG["glass_visible_transmittance_E"],
    g_transmittance=CONFIG["glass_visible_transmittance_E"],
    b_transmittance=CONFIG["glass_visible_transmittance_E"],
    refraction_index=1.52
)

glass_material_W = Glass(
    "GlassMaterialW",
    r_transmittance=CONFIG["glass_visible_transmittance_W"],
    g_transmittance=CONFIG["glass_visible_transmittance_W"],
    b_transmittance=CONFIG["glass_visible_transmittance_W"],
    refraction_index=1.52
)

glass_material_interior = Glass(
    "GlassMaterialInternal",
    r_transmittance=0.9,
    g_transmittance=0.9,
    b_transmittance=0.9,
    refraction_index=1.52
)

glass_material_skylight = Glass(
    "GlassMaterialSkylight",
    r_transmittance=CONFIG["glass_visible_transmittance_skylight"],
    g_transmittance=CONFIG["glass_visible_transmittance_skylight"],
    b_transmittance=CONFIG["glass_visible_transmittance_skylight"],
    refraction_index=1.52
)

air_wall_material = Glass(
    "AirWallMaterial",
    r_transmittance=1,
    g_transmittance=1,
    b_transmittance=1,
    refraction_index=1
)

wall_material = Plastic(
    "WallMaterial",
    r_reflectance=CONFIG["wall_reflectivity"],
    g_reflectance=CONFIG["wall_reflectivity"],
    b_reflectance=CONFIG["wall_reflectivity"],
    specularity=0,
    roughness=0
)

ceiling_material = Plastic(
    "CeilingMaterial",
    r_reflectance=CONFIG["ceiling_reflectivity"],
    g_reflectance=CONFIG["ceiling_reflectivity"],
    b_reflectance=CONFIG["ceiling_reflectivity"],
    specularity=0,
    roughness=0
)

floor_material = Plastic(
    "FloorMaterial",
    r_reflectance=CONFIG["floor_reflectivity"],
    g_reflectance=CONFIG["floor_reflectivity"],
    b_reflectance=CONFIG["floor_reflectivity"],
    specularity=0,
    roughness=0
)

In [28]:
HBAnalysisSurface

honeybee._hbanalysissurface.HBAnalysisSurface